<a href="https://colab.research.google.com/github/Nayanatharapmc/Amazon_Sentiment_Analysis/blob/main/NegativePhrasesExtractionLLama3Groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
# Getting Aspects of the Reviews using Llama3
***

In [1]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
GROQ_API_TOKEN = userdata.get('groq')

os.environ["GROQ_API_KEY"] = GROQ_API_TOKEN

In [3]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)



def llama3_70b(prompt, temperature=0.0, input_print=True):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="llama3-70b-8192",
      temperature=temperature,
  )

  return (chat_completion.choices[0].message.content)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
apple_df = pd.read_csv('/content/drive/MyDrive/neg_results.csv')
apple_df.head()

,Unnamed: 0,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,predictions
0,2,1.0,Malfunction a lot,I would not recommend buying any electronics f...,[],B07234GKN5,B07234GKN5,AFV4KKH356TLSZEHI3UBHN4SYSJQ,1582741702541,2,True,0
1,3,1.0,Battery is done for.,Battery is trashed essentially making the phon...,[],B07CQNF813,B07CQNF813,AFTPWGVZUDMJE5UUADHHZBQK4AIQ,1581536080691,0,True,0
2,7,1.0,Iphone 8+ no good,This iPhone 8+ was no good out the box. Scree...,[],B07759SBP7,B07759SBP7,AEDROFHP64B6OAVL42A4555ZNRVA,1567448437165,2,True,0
3,14,2.0,Not wat I was expecting.,Received wrong item.,[],B075NVNBCW,B075NVNBCW,AFZ4C3KAKQY7KJUISCKENRS4FWYA,1517779291469,7,True,0
4,19,1.0,Disappointed.,"This iPhone's screen has a delayed reaction, c...",[],B07759SBP7,B07759SBP7,AHS5L2SVSXFXN2AJHHJCO5AE3TZQ,1615484398335,1,True,0


In [7]:
reviews = apple_df['text'].to_list()

In [10]:
# Modified prompt to extract negative phrases directly
prompt = """
Extract all negative phrases from the given review delimited by triple quotes.
A negative phrase indicates a negative sentiment or defect in the product/service.
Answer with a list of phrases without any explanation. Answer in the format [\"phrase1\", \"phrase2\", ...]
without any explanation. If no negative phrases are found, then only answer \"[]\".
"""

# Step 1: Generate negative phrases
negative_phrases = []
for review in reviews:
    prompt_text = prompt + "'''" + review + "'''"
    output = llama3_70b(prompt_text)
    negative_phrases.append(eval(output))  # Evaluate the string to a list

# Step 2: Display the extracted negative phrases
for i, phrases in enumerate(negative_phrases):
    print(f"Review {i+1}:")
    for phrase in phrases:
        print(f"  - {phrase}")
pass

Review 1:
  - would not recommend
  - came defected
Review 2:
  - Battery is trashed
  - Complete waste of money
Review 3:
  - no good out the box
  - Screen Res was horrible
  - volume no good
Review 4:
  - Received wrong item.
Review 5:
  - delayed reaction
  - continues scrolling
  - places calls when I am not touching it
  - mutes me
  - Very frustrating
  - huge disappointment
Review 6:
  - My only negatives are
  - cannot see Private Browsing
  - people cannot hear me on speakerphone
  - these Smart Phones were made to get into your Business
  - it is not all that great to me
  - my last Apple product
  - they both were defective
  - over priced iPhone
Review 7:
  - my phone is super slow
  - The new IOS made it worse
  - This phone is so frustrating
Review 8:
  - many of the apps were not being supported anymore
  - This phone maxed out at iOS 7.1.2 and most apps are now 8.0 or higher
  - It freezes up and crashes often
  - it will only frustrate her
  - Not worth the extra $
Re

In [11]:
# Merge negative phrases with the apple_df
apple_df['negative_phrases'] = negative_phrases

In [12]:
apple_df.head()

,Unnamed: 0,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,predictions,negative_phrases
0,2,1.0,Malfunction a lot,I would not recommend buying any electronics f...,[],B07234GKN5,B07234GKN5,AFV4KKH356TLSZEHI3UBHN4SYSJQ,1582741702541,2,True,0,"[would not recommend, came defected]"
1,3,1.0,Battery is done for.,Battery is trashed essentially making the phon...,[],B07CQNF813,B07CQNF813,AFTPWGVZUDMJE5UUADHHZBQK4AIQ,1581536080691,0,True,0,"[Battery is trashed, Complete waste of money]"
2,7,1.0,Iphone 8+ no good,This iPhone 8+ was no good out the box. Scree...,[],B07759SBP7,B07759SBP7,AEDROFHP64B6OAVL42A4555ZNRVA,1567448437165,2,True,0,"[no good out the box, Screen Res was horrible,..."
3,14,2.0,Not wat I was expecting.,Received wrong item.,[],B075NVNBCW,B075NVNBCW,AFZ4C3KAKQY7KJUISCKENRS4FWYA,1517779291469,7,True,0,[Received wrong item.]
4,19,1.0,Disappointed.,"This iPhone's screen has a delayed reaction, c...",[],B07759SBP7,B07759SBP7,AHS5L2SVSXFXN2AJHHJCO5AE3TZQ,1615484398335,1,True,0,"[delayed reaction, continues scrolling, places..."


In [13]:
apple_df.to_csv('/content/drive/MyDrive/Apple_products_with_aspects_and_negative_phrases.csv')